In [1]:
!pip install dynet
!git clone https://github.com/neubig/nn4nlp-code.git

fatal: destination path 'nn4nlp-code' already exists and is not an empty directory.


In [0]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [0]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]


def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])


# Read in the data
train = list(read_dataset("nn4nlp-code/data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [0]:
# Start DyNet and defin trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

# Define the model
EMB_SIZE = 64
HID_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE))  # Word embeddings
fwdLSTM = dy.VanillaLSTMBuilder(1, EMB_SIZE, HID_SIZE, model)  # Forward RNN
bwdLSTM = dy.VanillaLSTMBuilder(1, EMB_SIZE, HID_SIZE, model)  # Backward RNN
W_sm = model.add_parameters((ntags, 2 * HID_SIZE))  # Softmax weights
b_sm = model.add_parameters((ntags))  # Softmax bias


# A function to calculate scores for one value
def calc_scores(words):
    dy.renew_cg()
    word_embs = [dy.lookup(W_emb, x) for x in words]
    fwd_init = fwdLSTM.initial_state()
    fwd_embs = fwd_init.transduce(word_embs)
    bwd_init = bwdLSTM.initial_state()
    bwd_embs = bwd_init.transduce(reversed(word_embs))
    W_sm_exp = dy.parameter(W_sm)
    b_sm_exp = dy.parameter(b_sm)
    return W_sm_exp * dy.concatenate([fwd_embs[-1], bwd_embs[-1]]) + b_sm_exp

In [2]:
for ITER in range(100):
    # Perform training
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += my_loss.value()
        my_loss.backward()
        trainer.update()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss / len(train), time.time() - start))
    # Perform training
    test_correct = 0.0
    for words, tag in dev:
        scores = calc_scores(words).npvalue()
        predict = np.argmax(scores)
        if predict == tag:
            test_correct += 1
    print("iter %r: test acc=%.4f" % (ITER, test_correct / len(dev)))

iter 0: train loss/sent=1.4910, time=36.87s
iter 0: test acc=0.3701
iter 1: train loss/sent=1.2269, time=37.10s
iter 1: test acc=0.4005
iter 2: train loss/sent=0.9949, time=36.94s
iter 2: test acc=0.4100
iter 3: train loss/sent=0.7487, time=36.94s
iter 3: test acc=0.3701
iter 4: train loss/sent=0.4808, time=37.06s
iter 4: test acc=0.3593
iter 5: train loss/sent=0.2781, time=36.73s
iter 5: test acc=0.3683
iter 6: train loss/sent=0.1411, time=36.27s
iter 6: test acc=0.3710
iter 7: train loss/sent=0.0636, time=36.31s
iter 7: test acc=0.3747
iter 8: train loss/sent=0.0331, time=36.28s
iter 8: test acc=0.3566
iter 9: train loss/sent=0.0234, time=36.50s
iter 9: test acc=0.3502
iter 10: train loss/sent=0.0152, time=36.47s
iter 10: test acc=0.3647
iter 11: train loss/sent=0.0084, time=36.34s
iter 11: test acc=0.3529
iter 12: train loss/sent=0.0097, time=36.43s
iter 12: test acc=0.3670
iter 13: train loss/sent=0.0108, time=36.46s
iter 13: test acc=0.3480
iter 14: train loss/sent=0.0143, time=36

iter 37: test acc=0.3416
iter 38: train loss/sent=0.0025, time=37.78s
iter 38: test acc=0.3425
iter 39: train loss/sent=0.0027, time=37.85s
iter 39: test acc=0.3548
iter 40: train loss/sent=0.0032, time=37.57s
iter 40: test acc=0.3507
iter 41: train loss/sent=0.0034, time=37.32s
iter 41: test acc=0.3480
iter 42: train loss/sent=0.0043, time=36.57s
iter 42: test acc=0.3566
iter 43: train loss/sent=0.0032, time=36.62s
iter 43: test acc=0.3643
iter 44: train loss/sent=0.0024, time=36.72s
iter 44: test acc=0.3511
iter 45: train loss/sent=0.0015, time=36.78s
iter 45: test acc=0.3656
iter 46: train loss/sent=0.0016, time=36.44s
iter 46: test acc=0.3489
iter 47: train loss/sent=0.0071, time=36.53s
iter 47: test acc=0.3615
iter 48: train loss/sent=0.0031, time=36.67s
iter 48: test acc=0.3552
iter 49: train loss/sent=0.0014, time=36.30s
iter 49: test acc=0.3570
iter 50: train loss/sent=0.0039, time=36.60s
iter 50: test acc=0.3507
iter 51: train loss/sent=0.0038, time=36.78s
iter 51: test acc=0.

iter 75: train loss/sent=0.0012, time=37.95s
iter 75: test acc=0.3715
iter 76: train loss/sent=0.0022, time=37.84s
iter 76: test acc=0.3715
iter 77: train loss/sent=0.0028, time=37.80s
iter 77: test acc=0.3588
iter 78: train loss/sent=0.0004, time=38.02s
iter 78: test acc=0.3538
iter 79: train loss/sent=0.0015, time=37.71s
iter 79: test acc=0.3516
iter 80: train loss/sent=0.0024, time=37.73s
iter 80: test acc=0.3633
iter 81: train loss/sent=0.0033, time=37.24s
iter 81: test acc=0.3543
iter 82: train loss/sent=0.0042, time=36.48s
iter 82: test acc=0.3602
iter 83: train loss/sent=0.0014, time=37.95s
iter 83: test acc=0.3570
iter 84: train loss/sent=0.0018, time=38.33s
iter 84: test acc=0.3643
iter 85: train loss/sent=0.0006, time=38.05s
iter 85: test acc=0.3543
iter 86: train loss/sent=0.0017, time=38.57s
iter 86: test acc=0.3566
iter 87: train loss/sent=0.0049, time=38.39s
iter 87: test acc=0.3579
iter 88: train loss/sent=0.0007, time=38.07s
iter 88: test acc=0.3611
iter 89: train loss/